神经网络的各种优化

In [5]:
import random
import sys
import numpy as np

二次 Cost

In [6]:
class QuadraticCost(object):
    def fn(a, y):
        """计算Cost的值"""
        return 0.5*np.linalg.norm(a-y)**2
    def delta(z, a, y):
        """求偏导"""
        return (a-y) * sigmoid_prime(z)

CrossEntropyCost函数,用来解决偏导值小时学习速率慢的问题

In [7]:
class CrossEntropyCost(object):
    def fn(a, y):
        """这里的负号其实可以提到外面"""
        """nan_to_num函数不了解"""
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))
    def delta(z, a, y):
        """决定学习速率快慢的因素"""
        return (a-y)

Network类

In [8]:
class Network(object):
    def __init__(self, sizes, cost=CrossEntropyCost):
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost=cost
        
    """下面的两个函数的区别在于有没有除以sqrt(x)"""
    def default_weight_initializer(self):
        #设置的默认biases参数为高斯分布 y行1列的向量
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        #设置的权重列数为前一个神经层的神经层个数,这里的参数除以了前层神经元个数的开方
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initializer(self):
        #这就是一个很普通的参数生成器
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a):
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0,                           
            evaluation_data=None,                  
            monitor_evaluation_cost=True,
            monitor_evaluation_accuracy=True,
            monitor_training_cost=True,
            monitor_training_accuracy=True,
            early_stopping_n = 0):
        best_accuracy=1

        training_data = list(training_data)
        n = len(training_data)

        if evaluation_data:                                         #处理验证集数据
            evaluation_data = list(evaluation_data)                 
            n_data = len(evaluation_data)

        # early stopping functionality:
        best_accuracy=0#为什么又设置为0了呢
        no_accuracy_change=0

        #误差与准确率的存储
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in range(0, n, mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(#eta是学习率，lmbda是什么呢
                    mini_batch, eta, lmbda, len(training_data))

            print("Epoch %s training complete" % j)
            print(monitor_training_cost)
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print("Cost on training data: {}".format(cost))
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data, convert=True)
                training_accuracy.append(accuracy)
                print("Accuracy on training data: {} / {}".format(accuracy, n))
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda, convert=True)
                evaluation_cost.append(cost)
                print("Cost on evaluation data: {}".format(cost))
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print("Accuracy on evaluation data: {} / {}".format(self.accuracy(evaluation_data), n_data))

            # Early stopping:
            if early_stopping_n > 0:
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    no_accuracy_change = 0
                    #print("Early-stopping: Best so far {}".format(best_accuracy))
                else:
                    no_accuracy_change += 1

                if (no_accuracy_change == early_stopping_n):
                    #print("Early-stopping: No accuracy change in last epochs: {}".format(early_stopping_n))
                    return evaluation_cost, evaluation_accuracy, training_cost, training_accuracy

        return evaluation_cost, evaluation_accuracy, \
            training_cost, training_accuracy

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            nabla_b = [nb+dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw+dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        """二次cost的随机梯度下降"""
        #w' = (1- η*λ/n)*w - η/len*(δC/δw)
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        activation = x
        activations = [x]
        zs = [] 
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        for l in range(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def accuracy(self, data, convert=False):
        if convert:
            results = [(np.argmax(self.feedforward(x)), np.argmax(y))
                       for (x, y) in data]
        else:
            results = [(np.argmax(self.feedforward(x)), y)
                        for (x, y) in data]

        result_accuracy = sum(int(x == y) for (x, y) in results)
        return result_accuracy

    def total_cost(self, data, lmbda, convert=False):
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if convert: y = vectorized_result(y)
            cost += self.cost.fn(a, y)/len(data)
            cost += 0.5*(lmbda/len(data))*sum(np.linalg.norm(w)**2 for w in self.weights) # '**' - to the power of.
        return cost

In [14]:
def vectorized_result(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e
def sigmoid(z):
    return 1.0/(1.0+np.exp(-z))
def sigmoid_prime(z):
    return sigmoid(z)*(1-sigmoid(z))

In [15]:
import mnist_loader
training_data,validation_data,test_data = mnist_loader.load_data_wrapper()
net = Network([784,30,10],cost = CrossEntropyCost)
net.SGD(training_data,4,10,0.1,lmbda = 5.0,evaluation_data = validation_data,monitor_evaluation_accuracy = True)

Epoch 0 training complete
True
Cost on training data: 1545.322871124418
Accuracy on training data: 46006 / 50000
Cost on evaluation data: 1545.294928953493
Accuracy on evaluation data: 9263 / 10000
Epoch 1 training complete
True
Cost on training data: 2128.8377229860007
Accuracy on training data: 46827 / 50000
Cost on evaluation data: 2128.8253354173776
Accuracy on evaluation data: 9404 / 10000
Epoch 2 training complete
True
Cost on training data: 2564.7658589862585
Accuracy on training data: 47251 / 50000
Cost on evaluation data: 2564.7630524114156
Accuracy on evaluation data: 9474 / 10000
Epoch 3 training complete
True
Cost on training data: 2914.2845278745212
Accuracy on training data: 47556 / 50000
Cost on evaluation data: 2914.2911294144374
Accuracy on evaluation data: 9514 / 10000


([1545.294928953493,
  2128.8253354173776,
  2564.7630524114156,
  2914.2911294144374],
 [9263, 9404, 9474, 9514],
 [1545.322871124418,
  2128.8377229860007,
  2564.7658589862585,
  2914.2845278745212],
 [46006, 46827, 47251, 47556])